In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MultiLabelBinarizer, FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, mean_squared_error
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.preprocessing import PolynomialFeatures
import shap
import matplotlib.pyplot as plt
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from collections import defaultdict

In [4]:
import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import pyarrow

file_name = "allo_cine"
version="2.2.1"

ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="72eb7803-e874-44cb-b6d9-33f2fa3eb88c",
    resource_group_name="vpoutotrg",
    workspace_name="mlstudio-groupe4"
)

data_asset = ml_client.data.get(file_name, version=version)
print(data_asset.path)


df = pd.read_parquet(data_asset.path)

df.head()

azureml://subscriptions/72eb7803-e874-44cb-b6d9-33f2fa3eb88c/resourcegroups/vpoutotrg/workspaces/mlstudio-groupe4/datastores/workspaceblobstore/paths/LocalUpload/19ec33ce8858cdb90c86accbe1c1a507/allocine_spider_clean.parquet


Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


,actors,critics_score,date,directors,editor,genre,langage,length,nationality,viewers_score,us_first_week_boxoffice,french_first_week_boxoffice,french_visa,title,vo_title,url
0,"[Arieh Worthalter, Arthur Harari, Stéphan Guér...",4.4,2023-09-27,[Cédric Kahn],Ad Vitam,"[Policier, Drame, Historique, Judiciaire]",[Français],116.0,[France],3.9,NaN,110011.0,157303,Le Procès Goldman,None,/article/fichearticle_gen_carticle=1000093547....
1,"[Monica Bellucci, Vincent Cassel, Albert Dupon...",3.5,2020-08-26,[Gaspar Noé],Carlotta Films,"[Drame, Thriller]","[Anglais, Français, Italien, Espagnol]",90.0,[France],NaN,NaN,2905.0,153336,Irréversible - Inversion Intégrale,None,/article/fichearticle_gen_carticle=18692477.html
2,[Thom Hoffman],2.8,2024-08-07,"[Richard Claus, Karsten Kiilerich]",Le Pacte,"[Aventure, Animation, Comédie, Famille]",[Néerlandais],84.0,"[Danemark, France, Allemagne, Pays-Bas]",3.0,NaN,39119.0,160622,Petit Panda en Afrique,Panda Bear in Africa,/article/fichearticle_gen_carticle=1000096064....
3,"[Lou de Laâge, Raphaël Personnaz, Isabelle Car...",3.3,2022-12-21,[Olivier Treiner],SND,[Drame],[Français],120.0,[France],3.8,NaN,64039.0,152607,Le Tourbillon de la vie,None,/article/fichearticle_gen_carticle=1000006465....
4,"[Michael B. Jordan, Jamie Foxx, Brie Larson, R...",3.0,2020-01-29,[Destin Daniel Cretton],Warner Bros. France,"[Biopic, Drame]",[Anglais],137.0,[U.S.A.],4.1,9713228.0,113153.0,152118,La Voie de la justice,Just Mercy,/video/player_gen_cmedia=19586793&cfilm=239735...


In [5]:
#datetime conversion
df['date'] = pd.to_datetime(df['date'], errors = 'coerce')

In [6]:

df['n_actors'] = df['actors'].apply(lambda x: len(x) if isinstance(x, np.ndarray) else 0)

In [7]:
df

,actors,critics_score,date,directors,editor,genre,langage,length,nationality,viewers_score,us_first_week_boxoffice,french_first_week_boxoffice,french_visa,title,vo_title,url,n_actors
0,"[Arieh Worthalter, Arthur Harari, Stéphan Guér...",4.4,2023-09-27,[Cédric Kahn],Ad Vitam,"[Policier, Drame, Historique, Judiciaire]",[Français],116.0,[France],3.9,NaN,110011.0,157303,Le Procès Goldman,None,/article/fichearticle_gen_carticle=1000093547....,16
1,"[Monica Bellucci, Vincent Cassel, Albert Dupon...",3.5,2020-08-26,[Gaspar Noé],Carlotta Films,"[Drame, Thriller]","[Anglais, Français, Italien, Espagnol]",90.0,[France],NaN,NaN,2905.0,153336,Irréversible - Inversion Intégrale,None,/article/fichearticle_gen_carticle=18692477.html,11
2,[Thom Hoffman],2.8,2024-08-07,"[Richard Claus, Karsten Kiilerich]",Le Pacte,"[Aventure, Animation, Comédie, Famille]",[Néerlandais],84.0,"[Danemark, France, Allemagne, Pays-Bas]",3.0,NaN,39119.0,160622,Petit Panda en Afrique,Panda Bear in Africa,/article/fichearticle_gen_carticle=1000096064....,1
3,"[Lou de Laâge, Raphaël Personnaz, Isabelle Car...",3.3,2022-12-21,[Olivier Treiner],SND,[Drame],[Français],120.0,[France],3.8,NaN,64039.0,152607,Le Tourbillon de la vie,None,/article/fichearticle_gen_carticle=1000006465....,9
4,"[Michael B. Jordan, Jamie Foxx, Brie Larson, R...",3.0,2020-01-29,[Destin Daniel Cretton],Warner Bros. France,"[Biopic, Drame]",[Anglais],137.0,[U.S.A.],4.1,9713228.0,113153.0,152118,La Voie de la justice,Just Mercy,/video/player_gen_cmedia=19586793&cfilm=239735...,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9153,"[Brad Pitt, Joey King, Aaron Taylor-Johnson, B...",3.2,2022-08-03,[David Leitch],Sony Pictures Releasing France,"[Action, Thriller]",[Anglais],127.0,[U.S.A.],3.9,30125000.0,491067.0,157053,Bullet Train,None,/video/player_gen_cmedia=19596933&cfilm=284846...,24
9154,"[Nicholas Hoult, J.K. Simmons, Kiefer Sutherla...",3.8,2024-10-30,[Clint Eastwood],Warner Bros. France,"[Drame, Thriller]",[Anglais],114.0,[U.S.A.],3.9,NaN,453813.0,-,Juré n°2,Juror #2,/video/player_gen_cmedia=20609934&cfilm=314868...,11
9155,"[Jacques Gamblin, Pascale Arbillot, Pablo Paul...",3.5,2022-05-11,[François Uzan],UGC Distribution,"[Comédie, Romance]",[Français],96.0,[France],2.8,NaN,37740.0,153600,On sourit pour la photo,None,/article/fichearticle_gen_carticle=18709697.html,10
9156,"[Alberto Ammann, Bruna Cusí, Ben Temple, Laura...",3.6,2024-05-01,"[Juan Sebastián Vásquez, Alejandro Rojas]",Condor Distribution,"[Drame, Thriller]",[Espagnol],77.0,[Espagne],3.8,NaN,44476.0,161784,Border Line,Upon Entry (La llegada),/video/player_gen_cmedia=20601439&cfilm=309465...,4


In [8]:
# Mapping acteur -> liste de boxoffices
actor_boxoffice = defaultdict(list)

# Itérer sur les lignes du DataFrame pour remplir le dictionnaire
for _, row in df.iterrows():
    box_office = row['french_first_week_boxoffice']
    if pd.isna(box_office):
        continue
    if isinstance(row['actors'], np.ndarray):
        for actor in row['actors']:
            actor_boxoffice[actor].append(box_office)


In [9]:
def get_actor_boxoffice_stats(actors, stat='mean'):
    if not isinstance(actors, (list, np.ndarray)):
        return np.nan
    values = []
    for actor in actors:
        if actor in actor_boxoffice:
            values.extend(actor_boxoffice[actor])
    if not values:
        return np.nan
    return np.mean(values) if stat == 'mean' else np.max(values)

# Moyenne des performances box-office précédentes des acteurs
df['actors_boxoffice_mean'] = df['actors'].apply(lambda x: get_actor_boxoffice_stats(x, stat='mean'))

# Max du box-office parmi les acteurs du film
df['actors_boxoffice_max'] = df['actors'].apply(lambda x: get_actor_boxoffice_stats(x, stat='max'))

In [10]:
df["date"] = pd.to_datetime(df["date"])
df["release_year"] = df["date"].dt.year
df["release_month"] = df["date"].dt.month


In [11]:
# Suppression de colonnes inutiles pour ce modèle
df_model = df.drop(columns=["actors", "title", "vo_title", "url", "date"])
df_model

,critics_score,directors,editor,genre,langage,length,nationality,viewers_score,us_first_week_boxoffice,french_first_week_boxoffice,french_visa,n_actors,actors_boxoffice_mean,actors_boxoffice_max,release_year,release_month
0,4.4,[Cédric Kahn],Ad Vitam,"[Policier, Drame, Historique, Judiciaire]",[Français],116.0,[France],3.9,NaN,110011.0,157303,16,1.271296e+05,714882.0,2023,9
1,3.5,[Gaspar Noé],Carlotta Films,"[Drame, Thriller]","[Anglais, Français, Italien, Espagnol]",90.0,[France],NaN,NaN,2905.0,153336,11,3.122669e+05,3685097.0,2020,8
2,2.8,"[Richard Claus, Karsten Kiilerich]",Le Pacte,"[Aventure, Animation, Comédie, Famille]",[Néerlandais],84.0,"[Danemark, France, Allemagne, Pays-Bas]",3.0,NaN,39119.0,160622,1,8.362500e+04,122090.0,2024,8
3,3.3,[Olivier Treiner],SND,[Drame],[Français],120.0,[France],3.8,NaN,64039.0,152607,9,1.474287e+05,1707373.0,2022,12
4,3.0,[Destin Daniel Cretton],Warner Bros. France,"[Biopic, Drame]",[Anglais],137.0,[U.S.A.],4.1,9713228.0,113153.0,152118,26,5.749542e+05,14800000.0,2020,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9153,3.2,[David Leitch],Sony Pictures Releasing France,"[Action, Thriller]",[Anglais],127.0,[U.S.A.],3.9,30125000.0,491067.0,157053,24,1.037413e+06,21840000.0,2022,8
9154,3.8,[Clint Eastwood],Warner Bros. France,"[Drame, Thriller]",[Anglais],114.0,[U.S.A.],3.9,NaN,453813.0,-,11,1.052801e+06,19002852.0,2024,10
9155,3.5,[François Uzan],UGC Distribution,"[Comédie, Romance]",[Français],96.0,[France],2.8,NaN,37740.0,153600,10,1.702827e+05,1806263.0,2022,5
9156,3.6,"[Juan Sebastián Vásquez, Alejandro Rojas]",Condor Distribution,"[Drame, Thriller]",[Espagnol],77.0,[Espagne],3.8,NaN,44476.0,161784,4,4.006833e+04,167270.0,2024,5


In [12]:
target = 'french_first_week_boxoffice'

In [13]:
df_model = df_model[df_model['french_first_week_boxoffice'] >= 10000]

In [14]:
df_model

,critics_score,directors,editor,genre,langage,length,nationality,viewers_score,us_first_week_boxoffice,french_first_week_boxoffice,french_visa,n_actors,actors_boxoffice_mean,actors_boxoffice_max,release_year,release_month
0,4.4,[Cédric Kahn],Ad Vitam,"[Policier, Drame, Historique, Judiciaire]",[Français],116.0,[France],3.9,NaN,110011.0,157303,16,1.271296e+05,714882.0,2023,9
2,2.8,"[Richard Claus, Karsten Kiilerich]",Le Pacte,"[Aventure, Animation, Comédie, Famille]",[Néerlandais],84.0,"[Danemark, France, Allemagne, Pays-Bas]",3.0,NaN,39119.0,160622,1,8.362500e+04,122090.0,2024,8
3,3.3,[Olivier Treiner],SND,[Drame],[Français],120.0,[France],3.8,NaN,64039.0,152607,9,1.474287e+05,1707373.0,2022,12
4,3.0,[Destin Daniel Cretton],Warner Bros. France,"[Biopic, Drame]",[Anglais],137.0,[U.S.A.],4.1,9713228.0,113153.0,152118,26,5.749542e+05,14800000.0,2020,1
5,3.9,[Sean Durkin],Metropolitan FilmExport,"[Biopic, Drame]",[Anglais],133.0,"[Grande-Bretagne, U.S.A.]",4.1,5044847.0,46627.0,161309,15,4.389270e+05,15075114.0,2024,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9153,3.2,[David Leitch],Sony Pictures Releasing France,"[Action, Thriller]",[Anglais],127.0,[U.S.A.],3.9,30125000.0,491067.0,157053,24,1.037413e+06,21840000.0,2022,8
9154,3.8,[Clint Eastwood],Warner Bros. France,"[Drame, Thriller]",[Anglais],114.0,[U.S.A.],3.9,NaN,453813.0,-,11,1.052801e+06,19002852.0,2024,10
9155,3.5,[François Uzan],UGC Distribution,"[Comédie, Romance]",[Français],96.0,[France],2.8,NaN,37740.0,153600,10,1.702827e+05,1806263.0,2022,5
9156,3.6,"[Juan Sebastián Vásquez, Alejandro Rojas]",Condor Distribution,"[Drame, Thriller]",[Espagnol],77.0,[Espagne],3.8,NaN,44476.0,161784,4,4.006833e+04,167270.0,2024,5


In [15]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6446 entries, 0 to 9157
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   critics_score                6445 non-null   float64
 1   directors                    6446 non-null   object 
 2   editor                       6446 non-null   object 
 3   genre                        6446 non-null   object 
 4   langage                      6446 non-null   object 
 5   length                       6443 non-null   float64
 6   nationality                  6445 non-null   object 
 7   viewers_score                6378 non-null   float64
 8   us_first_week_boxoffice      2297 non-null   float64
 9   french_first_week_boxoffice  6446 non-null   float64
 10  french_visa                  6446 non-null   object 
 11  n_actors                     6446 non-null   int64  
 12  actors_boxoffice_mean        6420 non-null   float64
 13  actors_boxoffice_max   

In [16]:
# 🧼 Séparer les features numériques / catégorielles
numeric_features = ['length', 'us_first_week_boxoffice',
                    'actors_boxoffice_mean', 'actors_boxoffice_max', 'n_actors', 'release_year', 'release_month']

categorical_features = ['directors', 'editor', 'genre', 'langage', 'nationality']

# ✅ Pipelines de prétraitement
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# ✨ Combine le tout
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# 🌟 Pipeline finale
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge(alpha=0.1))
])

for col in categorical_features:
    df_model[col] = df_model[col].astype(str)

# 🔀 Train-test split
df_model = df_model[df_model[target].notnull()]
X = df_model.drop(columns=[target])
y = df_model[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🧠 Entraînement
model.fit(X_train, y_train)

# 🔍 Prédictions et évaluation
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MAPE: {mape} %")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")




/tmp/ipykernel_242594/2694320286.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model[col] = df_model[col].astype(str)
/tmp/ipykernel_242594/2694320286.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model[col] = df_model[col].astype(str)
/tmp/ipykernel_242594/2694320286.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

MAE: 740547.5533019459
MAPE: 11.306319075114022 %
RMSE: 1410246.265264182
R²: 0.5829398838244493


/tmp/ipykernel_242594/2694320286.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model[col] = df_model[col].astype(str)


In [17]:
X_train_transformed = model.named_steps['preprocessor'].transform(X_train)


In [18]:
# Récupère les noms des colonnes numériques
num_cols = numeric_features

# Récupère les noms des colonnes après encodage OneHot
cat_encoder = model.named_steps['preprocessor'].named_transformers_['cat'].named_steps['encoder']
cat_cols = cat_encoder.get_feature_names_out(categorical_features)

# Combine les noms
feature_names = np.concatenate([num_cols, cat_cols])

# Et pour obtenir un DataFrame :
X_train_transformed_df = pd.DataFrame(X_train_transformed.toarray(), columns=feature_names)



In [19]:
X_train_transformed_df


,length,us_first_week_boxoffice,actors_boxoffice_mean,actors_boxoffice_max,n_actors,release_year,release_month,"directors_[""Billy O'Brien""]","directors_[""Gavin O'Connor""]","directors_[""Matthew O'Callaghan""]",...,nationality_['U.S.A.' 'Roumanie' 'Allemagne'],nationality_['U.S.A.' 'Russie'],nationality_['U.S.A.' 'République' 'tchèque'],nationality_['U.S.A.' 'Thaïlande'],nationality_['U.S.A.'],nationality_['Ukraine' 'Pays-Bas'],nationality_['Ukraine'],nationality_['Uruguay' 'Espagne' 'France' 'Argentine'],nationality_['Vénézuela' 'Argentine' 'Pérou' 'Allemagne'],nationality_['kazakhstan' 'Allemagne' 'Russie' 'Mongolie']
0,-1.123453,-0.185938,-0.376138,-0.698908,-0.965711,-0.918183,-0.717467,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.013489,-0.185938,0.352395,-0.587101,-1.298108,-1.216567,0.131636,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.910883,-0.896140,0.387232,0.478810,1.028675,-0.619800,-0.717467,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.848543,-0.185938,-0.318279,-0.662869,0.363880,-1.216567,-0.717467,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.023812,-0.185938,-0.352615,-0.658234,-0.633313,1.618077,1.546808,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5151,0.745937,-0.185938,-0.490586,-0.700083,-0.633313,-0.619800,-0.434433,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5152,-1.398363,-0.185938,-0.449716,-0.767585,-0.522514,-0.768992,-0.717467,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5153,0.416044,-0.185938,-0.239178,-0.528870,1.582671,-0.768992,0.980740,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5154,0.855901,-0.024188,-0.164619,-0.528870,0.585478,-1.514950,0.131636,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [21]:
df_model

,critics_score,directors,editor,genre,langage,length,nationality,viewers_score,us_first_week_boxoffice,french_first_week_boxoffice,french_visa,n_actors,actors_boxoffice_mean,actors_boxoffice_max,release_year,release_month
0,4.4,['Cédric Kahn'],Ad Vitam,['Policier' 'Drame' 'Historique' 'Judiciaire'],['Français'],116.0,['France'],3.9,NaN,110011.0,157303,16,1.271296e+05,714882.0,2023,9
2,2.8,['Richard Claus' 'Karsten Kiilerich'],Le Pacte,['Aventure' 'Animation' 'Comédie' 'Famille'],['Néerlandais'],84.0,['Danemark' 'France' 'Allemagne' 'Pays-Bas'],3.0,NaN,39119.0,160622,1,8.362500e+04,122090.0,2024,8
3,3.3,['Olivier Treiner'],SND,['Drame'],['Français'],120.0,['France'],3.8,NaN,64039.0,152607,9,1.474287e+05,1707373.0,2022,12
4,3.0,['Destin Daniel Cretton'],Warner Bros. France,['Biopic' 'Drame'],['Anglais'],137.0,['U.S.A.'],4.1,9713228.0,113153.0,152118,26,5.749542e+05,14800000.0,2020,1
5,3.9,['Sean Durkin'],Metropolitan FilmExport,['Biopic' 'Drame'],['Anglais'],133.0,['Grande-Bretagne' 'U.S.A.'],4.1,5044847.0,46627.0,161309,15,4.389270e+05,15075114.0,2024,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9153,3.2,['David Leitch'],Sony Pictures Releasing France,['Action' 'Thriller'],['Anglais'],127.0,['U.S.A.'],3.9,30125000.0,491067.0,157053,24,1.037413e+06,21840000.0,2022,8
9154,3.8,['Clint Eastwood'],Warner Bros. France,['Drame' 'Thriller'],['Anglais'],114.0,['U.S.A.'],3.9,NaN,453813.0,-,11,1.052801e+06,19002852.0,2024,10
9155,3.5,['François Uzan'],UGC Distribution,['Comédie' 'Romance'],['Français'],96.0,['France'],2.8,NaN,37740.0,153600,10,1.702827e+05,1806263.0,2022,5
9156,3.6,['Juan Sebastián Vásquez' 'Alejandro Rojas'],Condor Distribution,['Drame' 'Thriller'],['Espagnol'],77.0,['Espagne'],3.8,NaN,44476.0,161784,4,4.006833e+04,167270.0,2024,5


In [22]:
# 🧼 Séparer les features numériques / catégorielles
numeric_features = ['length', 'us_first_week_boxoffice',
                    'actors_boxoffice_mean', 'actors_boxoffice_max',
                    'n_actors', 'release_year', 'release_month']

categorical_features = ['directors', 'editor', 'genre', 'langage', 'nationality']

# ✅ Pipelines de prétraitement
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# ✨ Combine le tout
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# 🌟 Pipeline de base
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
])

# 🧪 Paramètres à tester
param_grid = {
    'regressor__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

# 🧽 Préparation des données
for col in categorical_features:
    df_model[col] = df_model[col].astype(str)

df_model = df_model[df_model[target].notnull()]
X = df_model.drop(columns=[target])
y = df_model[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔍 GridSearch
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid.fit(X_train, y_train)

# ✅ Résultats
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

print(f"Best alpha: {grid.best_params_['regressor__alpha']}")
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred), "%")
print("RMSE:", root_mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))


Best alpha: 100
MAE: 449233.7718991459
MAPE: 5.903669121664958 %
RMSE: 1088009.926483058
R²: 0.7517584589103642


In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Transformer numérique avec polynômes
numeric_transformer_poly = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('poly', PolynomialFeatures(degree=2, include_bias=False))
    ('scaler', StandardScaler())
])

# Reste identique
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Nouveau préprocessor
preprocessor_poly = ColumnTransformer(transformers=[
    ('num', numeric_transformer_poly, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Pipeline avec Ridge
pipe_poly = Pipeline(steps=[
    ('preprocessor', preprocessor_poly),
    ('regressor', Ridge())
])

# GridSearch
param_grid_poly = {
    'regressor__alpha': [0.01, 0.1, 1, 10]
}

grid_poly = GridSearchCV(pipe_poly, param_grid_poly, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_poly.fit(X_train, y_train)

# Résultats
y_pred_poly = grid_poly.best_estimator_.predict(X_test)
print(f"\n🔷 PolynomialFeatures (degree=2) — Best alpha: {grid_poly.best_params_['regressor__alpha']}")
print("MAE:", mean_absolute_error(y_test, y_pred_poly))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred_poly))
print("RMSE:", root_mean_squared_error(y_test, y_pred_poly))
print("R²:", r2_score(y_test, y_pred_poly))



🔷 PolynomialFeatures (degree=2) — Best alpha: 10
MAE: 456194.1772034135
MAPE: 5.298982422978008
RMSE: 915111.825265258
R²: 0.8243868317103111


# Split datas by years of release


In [32]:
# Split temporel
train_df = df_model[df_model["release_year"] <= 2022]
test_df = df_model[df_model["release_year"] >= 2023]

# Séparation X / y
X_train = train_df.drop(columns=[target])
y_train = train_df[target]

X_test = test_df.drop(columns=[target])
y_test = test_df[target]


In [33]:
from sklearn.preprocessing import PolynomialFeatures

# Transformer numérique avec polynômes
numeric_transformer_poly = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())
])

# Reste identique
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Nouveau préprocessor
preprocessor_poly = ColumnTransformer(transformers=[
    ('num', numeric_transformer_poly, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Pipeline avec Ridge
pipe_poly = Pipeline(steps=[
    ('preprocessor', preprocessor_poly),
    ('regressor', Ridge())
])

# GridSearch
param_grid_poly = {
    'regressor__alpha': [0.01, 0.1, 1, 10]
}

grid_poly = GridSearchCV(pipe_poly, param_grid_poly, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_poly.fit(X_train, y_train)

# Résultats
y_pred_poly = grid_poly.best_estimator_.predict(X_test)
print(f"\n🔷 PolynomialFeatures (degree=2) — Best alpha: {grid_poly.best_params_['regressor__alpha']}")
print("MAE:", mean_absolute_error(y_test, y_pred_poly))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred_poly))
print("RMSE:", root_mean_squared_error(y_test, y_pred_poly))
print("R²:", r2_score(y_test, y_pred_poly))



🔷 PolynomialFeatures (degree=2) — Best alpha: 10
MAE: 265933.4208836473
MAPE: 4.5041750034762735
RMSE: 649866.3877010809
R²: 0.6597693951026791


R² is smaller but the RMSE decreases, so it seems to be a great solution

# Test by splitting by nationality

In [34]:
df_model['french_prod']= df_model['nationality'].apply(lambda x: 1 if "France" in x else 0)
df_model['usa_prod']= df_model['nationality'].apply(lambda x: 1 if "U.S.A." in x else 0)
df_model


,critics_score,directors,editor,genre,langage,length,nationality,viewers_score,us_first_week_boxoffice,french_first_week_boxoffice,french_visa,n_actors,actors_boxoffice_mean,actors_boxoffice_max,release_year,release_month,french_prod,usa_prod
0,4.4,['Cédric Kahn'],Ad Vitam,['Policier' 'Drame' 'Historique' 'Judiciaire'],['Français'],116.0,['France'],3.9,NaN,110011.0,157303,16,1.271296e+05,714882.0,2023,9,1,0
2,2.8,['Richard Claus' 'Karsten Kiilerich'],Le Pacte,['Aventure' 'Animation' 'Comédie' 'Famille'],['Néerlandais'],84.0,['Danemark' 'France' 'Allemagne' 'Pays-Bas'],3.0,NaN,39119.0,160622,1,8.362500e+04,122090.0,2024,8,1,0
3,3.3,['Olivier Treiner'],SND,['Drame'],['Français'],120.0,['France'],3.8,NaN,64039.0,152607,9,1.474287e+05,1707373.0,2022,12,1,0
4,3.0,['Destin Daniel Cretton'],Warner Bros. France,['Biopic' 'Drame'],['Anglais'],137.0,['U.S.A.'],4.1,9713228.0,113153.0,152118,26,5.749542e+05,14800000.0,2020,1,0,1
5,3.9,['Sean Durkin'],Metropolitan FilmExport,['Biopic' 'Drame'],['Anglais'],133.0,['Grande-Bretagne' 'U.S.A.'],4.1,5044847.0,46627.0,161309,15,4.389270e+05,15075114.0,2024,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9153,3.2,['David Leitch'],Sony Pictures Releasing France,['Action' 'Thriller'],['Anglais'],127.0,['U.S.A.'],3.9,30125000.0,491067.0,157053,24,1.037413e+06,21840000.0,2022,8,0,1
9154,3.8,['Clint Eastwood'],Warner Bros. France,['Drame' 'Thriller'],['Anglais'],114.0,['U.S.A.'],3.9,NaN,453813.0,-,11,1.052801e+06,19002852.0,2024,10,0,1
9155,3.5,['François Uzan'],UGC Distribution,['Comédie' 'Romance'],['Français'],96.0,['France'],2.8,NaN,37740.0,153600,10,1.702827e+05,1806263.0,2022,5,1,0
9156,3.6,['Juan Sebastián Vásquez' 'Alejandro Rojas'],Condor Distribution,['Drame' 'Thriller'],['Espagnol'],77.0,['Espagne'],3.8,NaN,44476.0,161784,4,4.006833e+04,167270.0,2024,5,0,0


In [39]:
numeric_features = ['length', 'us_first_week_boxoffice',
                    'actors_boxoffice_mean', 'actors_boxoffice_max',
                    'n_actors', 'release_year', 'release_month']

categorical_features = ['directors', 'editor', 'genre', 'langage', 'nationality', 'french_prod', 'usa_prod']

train_df = df_model[df_model["release_year"] <= 2022]
test_df = df_model[df_model["release_year"] >= 2023]

# Séparation X / y
X_train = train_df.drop(columns=[target])
y_train = train_df[target]

X_test = test_df.drop(columns=[target])
y_test = test_df[target]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_transformer_poly = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())
])

# Reste identique
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Nouveau préprocessor
preprocessor_poly = ColumnTransformer(transformers=[
    ('num', numeric_transformer_poly, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Pipeline avec Ridge
pipe_poly = Pipeline(steps=[
    ('preprocessor', preprocessor_poly),
    ('regressor', Ridge())
])

# GridSearch
param_grid_poly = {
    'regressor__alpha': [0.01, 0.1, 1, 10]
}

grid_poly = GridSearchCV(pipe_poly, param_grid_poly, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_poly.fit(X_train, y_train)

# Résultats
y_pred_poly = grid_poly.best_estimator_.predict(X_test)
print(f"\n🔷 PolynomialFeatures (degree=2) — Best alpha: {grid_poly.best_params_['regressor__alpha']}")
print("MAE:", mean_absolute_error(y_test, y_pred_poly))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred_poly))
print("RMSE:", root_mean_squared_error(y_test, y_pred_poly))
print("R²:", r2_score(y_test, y_pred_poly))



🔷 PolynomialFeatures (degree=2) — Best alpha: 10
MAE: 265909.13509373495
MAPE: 4.489554853395522
RMSE: 649688.6429132783
R²: 0.6599554824205403


In [42]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score, mean_squared_error
import numpy as np

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

numeric_features = ['length', 'us_first_week_boxoffice',
                    'actors_boxoffice_mean', 'actors_boxoffice_max',
                    'n_actors', 'release_year', 'release_month']

categorical_features = ['directors', 'editor', 'genre', 'langage', 'nationality', 'french_prod', 'usa_prod']

# 🔹 Split temporel
train_df = df_model[df_model["release_year"] <= 2022]
test_df = df_model[df_model["release_year"] >= 2023]

# 🔹 Séparation X / y pour test
X_test = test_df.drop(columns=[target])
y_test = test_df[target]

# 🔹 Séparation X / y pour train
X_full_train = train_df.drop(columns=[target])
y_full_train = train_df[target]

# ✅ Stratification sur 'french_prod' dans le split d'entraînement
X_train, X_valid, y_train, y_valid = train_test_split(
    X_full_train, y_full_train,
    test_size=0.2,
    random_state=42,
    stratify=X_full_train["french_prod"]
)

# 💡 Prétraitement numérique avec PolynomialFeatures
numeric_transformer_poly = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())
])

# 💡 Prétraitement catégoriel
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# 💡 Préprocesseur complet
preprocessor_poly = ColumnTransformer(transformers=[
    ('num', numeric_transformer_poly, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# 🔧 Pipeline avec Ridge
pipe_poly = Pipeline(steps=[
    ('preprocessor', preprocessor_poly),
    ('regressor', Ridge())
])

# 🔍 GridSearch
param_grid_poly = {
    'regressor__alpha': [0.01, 0.1, 1, 10]
}

grid_poly = GridSearchCV(pipe_poly, param_grid_poly, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_poly.fit(X_train, y_train)

# ✅ Évaluation sur validation
y_pred_valid = grid_poly.best_estimator_.predict(X_valid)
print("\n🔍 Évaluation sur le set de validation (stratifié):")
print("MAE:", mean_absolute_error(y_valid, y_pred_valid))
print("MAPE:", mean_absolute_percentage_error(y_valid, y_pred_valid))
print("RMSE:", root_mean_squared_error(y_valid, y_pred_valid))
print("R²:", r2_score(y_valid, y_pred_valid))

# ✅ Évaluation sur le test (2023+)
y_pred_test = grid_poly.best_estimator_.predict(X_test)
print(f"\n🔷 PolynomialFeatures (degree=2) — Best alpha: {grid_poly.best_params_['regressor__alpha']}")
print("MAE:", mean_absolute_error(y_test, y_pred_test))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred_test))
print("RMSE:", root_mean_squared_error(y_test, y_pred_test))
print("R²:", r2_score(y_test, y_pred_test))



🔍 Évaluation sur le set de validation (stratifié):
MAE: 451319.58479618875
MAPE: 4.986735502073317
RMSE: 905725.2054717591
R²: 0.6923478574087476

🔷 PolynomialFeatures (degree=2) — Best alpha: 10
MAE: 267975.2882994271
MAPE: 4.579181166589385
RMSE: 648212.8550747376
R²: 0.6614985711103198


In [43]:
# 🧼 Séparer les features numériques / catégorielles
numeric_features = ['length', 'us_first_week_boxoffice',
                    'actors_boxoffice_mean', 'actors_boxoffice_max',
                    'n_actors', 'release_year', 'release_month']

categorical_features = ['directors', 'editor', 'genre', 'langage', 'nationality', 'french_prod', 'usa_prod']

# ✅ Pipelines de prétraitement
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# ✨ Combine le tout
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# 🌟 Pipeline de base
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
])

# 🧪 Paramètres à tester
param_grid = {
    'regressor__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

# 🧽 Préparation des données
for col in categorical_features:
    df_model[col] = df_model[col].astype(str)

df_model = df_model[df_model[target].notnull()]
X = df_model.drop(columns=[target])
y = df_model[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_model['french_prod'])

# 🔍 GridSearch
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid.fit(X_train, y_train)

# ✅ Résultats
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

print(f"Best alpha: {grid.best_params_['regressor__alpha']}")
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred), "%")
print("RMSE:", root_mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))


Best alpha: 100
MAE: 426635.6336106106
MAPE: 5.391010269180373 %
RMSE: 938708.5785728593
R²: 0.7759895032633191


In [44]:
from sklearn.model_selection import StratifiedShuffleSplit

def time_based_split(df, test_years=3, test_ratio=0.5):
    """
    Crée un découpage train/test basé sur le temps, avec stratification
    sur les colonnes 'french_prod' et 'french_first_week_boxoffice'.
    
    Args:
        df: DataFrame avec les données
        test_years: Nombre d'années récentes à considérer pour le découpage
        test_ratio: Proportion des films des années récentes à mettre dans le test set
    
    Returns:
        indices_train, indices_test
    """
    # S'assurer que la colonne date est bien en datetime
    df["date"] = pd.to_datetime(df["date"])
    
    # Déterminer la date limite pour les années récentes
    max_date = df["date"].max()
    cutoff_date = pd.Timestamp(year=max_date.year - test_years, month=1, day=1)
    
    # Masque pour les films récents
    recent_mask = df["date"] >= cutoff_date
    old_mask = ~recent_mask

    # Films anciens => 100% train
    old_indices = df[old_mask].index.tolist()
    
    # Films récents à splitter avec stratification
    recent_df = df[recent_mask].copy()
    
    # On crée une variable combinée pour la stratification
    recent_df["stratify_col"] = recent_df["french_prod"].astype(str) + "_" + pd.qcut(
        recent_df["french_first_week_boxoffice"], q=4, duplicates='drop'
    ).astype(str)

    # Appliquer le StratifiedShuffleSplit
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=test_ratio, random_state=42)
    split = next(splitter.split(recent_df, recent_df["stratify_col"]))
    recent_train_idx, recent_test_idx = split

    # Récupérer les vrais indices du DataFrame original
    recent_indices = recent_df.index.tolist()
    test_indices = [recent_indices[i] for i in recent_test_idx]
    train_indices = old_indices + [recent_indices[i] for i in recent_train_idx]

    # Affichage
    print(f"Découpage temporel stratifié : {len(train_indices)} films pour l'entraînement, {len(test_indices)} films pour le test")
    print(f"Plage de dates d'entraînement : {df.loc[train_indices, 'date'].min()} à {df.loc[train_indices, 'date'].max()}")
    print(f"Plage de dates de test : {df.loc[test_indices, 'date'].min()} à {df.loc[test_indices, 'date'].max()}")

    return train_indices, test_indices

In [46]:
df['french_prod']= df['nationality'].apply(lambda x: 1 if "France" in x else 0)
df['usa_prod']= df['nationality'].apply(lambda x: 1 if "U.S.A." in x else 0)
df

test_years=3
test_ratio=0.5
train_indices, test_indices = time_based_split(df, test_years=test_years, test_ratio=test_ratio)
X_train, X_test = X.loc[train_indices], X.loc[test_indices]
y_train, y_test = y.loc[train_indices], y.loc[test_indices]

TypeError: argument of type 'NoneType' is not iterable

In [48]:
df.isna().sum()

actors                           86
critics_score                    13
date                              0
directors                         0
editor                            0
genre                             0
langage                           0
length                           12
nationality                       6
viewers_score                   398
us_first_week_boxoffice        6787
french_first_week_boxoffice       3
french_visa                       0
title                             0
vo_title                       5632
url                               0
n_actors                          0
actors_boxoffice_mean            86
actors_boxoffice_max             86
release_year                      0
release_month                     0
dtype: int64